In [236]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import folium
from folium import plugins
import re
from pygeocoder import Geocoder
import json

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ramitgoolry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ramitgoolry/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Reading Data

In [237]:
df = pd.read_csv('Datasets/pd_collisions_datasd_v1.csv')

In [238]:
df.head()

,report_id,date_time,police_beat,address_number_primary,address_pd_primary,address_road_primary,address_sfx_primary,address_pd_intersecting,address_name_intersecting,address_sfx_intersecting,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl
0,170082,2017-01-01 00:01:00,935,5500,,VALERIO,TRAIL,,,,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
1,170101,2017-01-01 00:01:00,322,6400,,CRAWFORD,STREET,,,,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
2,170166,2017-01-01 00:01:00,124,8300,,CAM DEL ORO,,,,,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
3,170218,2017-01-01 00:01:00,325,8100,,ROYAL GORGE,DRIVE,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,MISDEMEANOR
4,170097,2017-01-01 01:00:00,521,1000,,11TH,AVENUE,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,MISDEMEANOR


# Extraction of Locations from Dataset

In [239]:
addresses = df.apply(lambda x : ' '.join(
    str(i).strip() for i in [x.address_number_primary, x.address_pd_primary, x.address_road_primary, x.address_sfx_primary] 
) + ', SAN DIEGO', axis = 1)

In [240]:
addresses.head()

0        5500  VALERIO TRAIL, SAN DIEGO
1      6400  CRAWFORD STREET, SAN DIEGO
2         8300  CAM DEL ORO , SAN DIEGO
3    8100  ROYAL GORGE DRIVE, SAN DIEGO
4          1000  11TH AVENUE, SAN DIEGO
dtype: object

## Geocoding

In [241]:
API_dict = json.load(op'API.json')
coder = Geocoder(API_dict['API_KEY'])

locations = addresses[:100].apply(lambda address : coder.geocode(address).coordinates)

AttributeError: 'str' object has no attribute 'read'

# Map

In [232]:
# Creating map
m = folium.Map(center, zoom_start = 11)

### Mark each point

In [233]:
for location in locations:
    folium.CircleMarker([location[0], location[1]],
                        radius=15,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)

### Creating Heatmap

In [234]:
points_df = pd.DataFrame({
    'latitude' : list(locations.apply(lambda p : p[0])),
    'longitude' : list(locations.apply(lambda p : p[1]))
})

m.add_children(plugins.HeatMap(points_df.values, radius = 20))
m